## Generating a 3D Surface Mesh from a Point Cloud

In [1]:
import numpy as np
import laspy
from open3d import PointCloud, Vector3dVector, estimate_normals, KDTreeSearchParamHybrid, draw_geometries, \
                   voxel_down_sample, orient_normals_to_align_with_direction
from scipy.spatial import cKDTree
np.set_printoptions(suppress=True)

#### 1. Read in a LAZ point cloud file using `laspy`
Convert it to a 3D `numpy` array.

In [2]:
lasfile = laspy.file.File('../tests/sample1_classified.laz')

In [3]:
class_ids, counts = np.unique(lasfile.classification, return_counts=True)
for class_id, count in zip(class_ids, counts):
    print('Lidar classification {}: {:0,} points'.format(class_id, count))

Lidar classification 1: 166,445 points
Lidar classification 2: 58,429 points
Lidar classification 5: 412,167 points
Lidar classification 6: 26,818 points


In [4]:
xyz = np.stack((lasfile.x, lasfile.y, lasfile.z), axis=-1)
xyz.shape

(663859, 3)

In [5]:
# let's ditch ground and unclassified points (keeping only buildings and high vegetation)
good_points_idx = np.any([lasfile.classification == 6, lasfile.classification == 5], axis=0)
xyz = xyz[good_points_idx]

#### 2. Convert the array into an `open3d` point cloud and estimate normals

In [6]:
pcd = PointCloud()
pcd.points = Vector3dVector(xyz)

# estimate surface normals for each point in the point cloud
estimate_normals(pcd, search_param=KDTreeSearchParamHybrid(radius=5, max_nn=30))
orient_normals_to_align_with_direction(pcd) # defaults to orientation with positive z up

True

In [7]:
# downsample for visualization
down_pcd = voxel_down_sample(pcd, voxel_size = 2)
draw_geometries([down_pcd])

#### 3. Create a 3D grid with numpy

In [ ]:
def make_grid3D(xyz, buffers=(0,0,0), steps=(1.0, 1.0, 1.0)):
    """Given a point cloud, generates a enclosing 3D grid.
    
    Parameters
    ----------
    xyz: array with shape (n, 3)
        3-D coordinates of n points in a point cloud
        [[x1, y1, z1],[x2, y2, z2],...,[xn, yn, zn]]
    
    buffers: tuple
        Extra distance to buffer each dimension. Defaults
        to (0, 0, 0) for no extra buffer. Can be specified
        as 3-tuple to indicate equal buffers to the min and max 
        of each dimension, such as:
            
            buffer=(x_buffer, y_buffer, z_buffer)
            
        or with different buffers to the min and max of each 
        dimension, such as:
            
            buffer=((x_min_buffer, y_min_buffer, z_min_buffer),
                    x_max_buffer, y_max_buffer, z_max_buffer))
    
    step: 3-tuple (x_step, y_step, z_step)
        Step-size/resolution to use for constructing grid.
        Defaults to (1.0, 1.0, 1.0)
    
    Returns
    -------
    meshgrid, shape
    """
    buffers = np.asarray(buffers)
    
    if buffers.shape == (3,) or buffer.shape == (1,3):
        min_buffs = max_buffs = buffers
    elif buffers.shape == (2,3):
        min_buffs, max_buffs = buffers
    else:
        raise TypeError('Buffers is not an appropriate shape')
    
    x_min, y_min, z_min = xyz.min(axis=0) - min_buffs
    x_max, y_max, z_max = xyz.max(axis=0) + max_buffs
    x_step, y_step, z_step = steps
    
    return np.mgrid[x_min:x_max:x_step, 
                    y_min:y_max:y_step, 
                    z_min:z_max:z_step]

def angle_between_2v(vector):
    """ Calculates the angle between two unit vectors that 
    have been stacked into a single vector.
    
    Parameters
    ----------
    vector: array with shape (n,6)
        An array containing two unit vectors [x1,y1,z1,x2,y2,z2]
        for n points
        
    Returns
    -------
    radians: array with shape (n,) 
        An array containing the angle between the two vectors,
        in radians
    """
    return np.arccos(np.clip(np.dot(vector[0:3], vector[3:6]), -1.0, 1.0))

def angle_between(vec1, vec2):
    """ Calculates the angle between two vectors.
    
    Acute angles indicate unit vectors point in
    similar directions, while obtuse angles indicate
    they point in opposite directions. In the case where 
    one of the vectors is a surface normal, an acute angle
    would indicate the other point which was used to calculate
    the other unit vector is beneath or inside the surface. 
    
    Parameters
    ----------
    vec1: array with shape (n,3)
        unit vector for n samples
    vec2: array with shape (n,3)
        unit vector for n samples
        
    Returns
    -------
    radians: array with shape (n,)
        The angle between the two unit vectors, in radians
    """
    # combine the two vectors into a single vector with shape (n,6)
    vector = np.hstack((vec1, vec2))
    
    return np.apply_along_axis(angle_between_2v, 1, vector)

In [ ]:
xx, yy, zz = make_grid3D(np.asarray(pcd.points), buffers=(10,10,10))
grid_pts = np.asarray((xx.flatten(), yy.flatten(), zz.flatten())).T

In [ ]:
grid_pts.shape

In [ ]:
grid_pcd = PointCloud()
grid_pcd.points = Vector3dVector(grid_pts)

In [ ]:
# draw_geometries([grid_pcd])

In [ ]:
kdtree = cKDTree(pcd.points)

In [ ]:
# compute the euclidean distance between each grid point and the nearest point in the point cloud
distances, neighbors = kdtree.query(grid_pts, n_jobs=-1)

In [ ]:
pcd_normals = np.asarray(pcd.normals)

In [ ]:
# calculate vectors that represent the shifts in x, y, and z dimensions
# to get from the nearest point in the point cloud to a grid point
grid_pt_vectors = grid_pts - np.asarray(pcd.points)[neighbors]

# by indicating the distances vector is 1-dimensional, 
# we can calculate the unit vectors, otherwise numpy will have 
# trouble broadcasting vectors and distances together
distances.shape = (distances.shape[0],1)

# divide the vectors by the distance between the points to get unit vectors
grid_pt_unit_vectors = grid_pt_vectors / distances

In [ ]:
# calculate the angles between the grid points and the nearest neighbor 
# in the point cloud by calculating the difference between the estimated
# normal for the point in the point cloud (from open3d) indicating the
# direction between the point in the point cloud and the grid point
angles = angle_between(grid_pt_unit_vectors, pcd_normals[neighbors])

In [ ]:
np.degrees(angles[0])